In [ ]:
import os
import pandas as pd

from core.EvaluatorForBIRD import EvaluatorForBIRD
from agents.ZeroShotAgent import ZeroShotAgent
from agents.OptimizerAgent import OptimizerAgent
from agents.MultiAgentDiscussion import MultiAgentDiscussion

from config import MODEL, EXPERIMENT, OUTPUT_PATH
from core.Model import LLM, GenerationConfig
from utils import dump_to_json, read_dataset, get_databases, deploy_ollama
import api_keys

In [1]:
import re
from core.Model import SupportedModels, GenerationConfig, LLM


llm = LLM(SupportedModels.Ollama.llama_32_1b)
cfg = GenerationConfig()

/home/fahim/Documents/sql-gen/core/Model.py:210: UserWarning: Debug mode OFF: LLM will generate response
  warnings.warn("Debug mode OFF: LLM will generate response")


Pulling ollama model llama3.2:1b...
pulling manifest


pulling 74701a8c35f6: 100%|██████████| 1.32G/1.32G [00:00<00:00, 7.38TB/s]
pulling 966de95ca8a6: 100%|██████████| 1.43k/1.43k [00:00<00:00, 12.0MB/s]
pulling fcc5a6bec9da: 100%|██████████| 7.71k/7.71k [00:00<00:00, 47.5MB/s]
pulling a70ff7e570d9: 100%|██████████| 6.02k/6.02k [00:00<00:00, 55.2MB/s]
pulling 4f659a1e86d7: 100%|██████████| 485/485 [00:00<00:00, 2.41MB/s]

verifying sha256 digest
writing manifest
success
Successfully pulled model llama3.2:1b
Client Instantiated: Ollama model llama3.2:1b active at http://localhost:11434/ (status_code=200)


In [ ]:
a = """
Here is your code:

'''python
q1 = "SELECT T1.CustomerID FROM customers AS T1 INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Segment = 'LAM' AND SUBSTR(T2.Date, 1, 4) = '2012' GROUP BY T1.CustomerID ORDER BY SUM(T2.Consumption) ASC LIMIT 1"
rows = cursor.execute(q1).fetch all
'''

Does this answer you query?
"""

from agents.OptimizerAgent import OptimizerAgent
from core.Model import LLM, SupportedModels, GenerationConfig

llm = LLM(SupportedModels.Ollama.llama_32_1b)
op_agent = OptimizerAgent(llm, None, None)


In [25]:
a = """
Here is your code:

q1 = "SELECT T1.CustomerID FROM customers AS T1 INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Segment = 'LAM' AND SUBSTR(T2.Date, 1, 4) = '2012' GROUP BY T1.CustomerID ORDER BY SUM(T2.Consumption) ASC LIMIT 1"

q2 = "SELECT AVG(T2.Consumption) / 12 FROM customers AS T1 INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID WHERE SUBSTR(T2.Date, 1, 4) = '2013' AND T1.Segment = 'SME'"

Does this answer you query?
"""
def make_msg(system_prompt, user_prompt):
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

def match_sql_regex(response):
    sql = re.search(r'```sql(.*?)```', response, re.DOTALL)
    if sql:
        return sql.group(1).strip()
    else:
        return ''

sql = match_sql_regex(a)
if not sql:
    system_prompt = f"Please extract and enclose the SQL query from the text within a```sql <<your response here>>''' tag"
    llm_parsed = llm(make_msg(system_prompt, f"### Text:\n{a}"), GenerationConfig(temperature=0.4))
    print('llm:\n', llm_parsed)
    parsed = match_sql_regex(llm_parsed)
    print('\n\nfixed:\n',parsed)



llm:
 ```sql
SELECT 
    ( SELECT SUM(T2.Consumption)
     FROM customers AS T1
     INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID
     WHERE T1.Segment = 'LAM'
         AND SUBSTR(T2.Date, 1, 4) = '2012'
         GROUP BY T1.CustomerID
     ) AS Q1,
    ( SELECT AVG(T2.Consumption)
     FROM customers AS T1
     INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID
     WHERE SUBSTR(T2.Date, 1, 4) = '2013'
         AND T1.Segment = 'SME' ) AS Q2
```


fixed:
 SELECT 
    ( SELECT SUM(T2.Consumption)
     FROM customers AS T1
     INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID
     WHERE T1.Segment = 'LAM'
         AND SUBSTR(T2.Date, 1, 4) = '2012'
         GROUP BY T1.CustomerID
     ) AS Q1,
    ( SELECT AVG(T2.Consumption)
     FROM customers AS T1
     INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID
     WHERE SUBSTR(T2.Date, 1, 4) = '2013'
         AND T1.Segment = 'SME' ) AS Q2


In [ ]:
from tqdm import tqdm
import ollama
from core.Model import SupportedModels

model = 'llama3.2:3b'
client = ollama.Client(host='http://localhost:11434/', timeout=60) 

def opull():
    print(f"Pulling ollama model {model}...")
    current_digest, bars = '', {}
    for progress in client.pull(model, stream=True):
        digest = progress.get('digest', '')
        if digest != current_digest and current_digest in bars:
            bars[current_digest].close()

        if not digest:
            print(progress.get('status'))
            continue

        if digest not in bars and (total := progress.get('total')):
            bars[digest] = tqdm(total=total, desc=f'pulling {digest[7:19]}', unit='B', unit_scale=True)

        if completed := progress.get('completed'):
            bars[digest].update(completed - bars[digest].n)

        current_digest = digest
    print(f"Successfully pulled model {model}")


try:
    opull()
except ollama.ResponseError as e:
    raise ValueError(f"Ollama pull() failed: ({e.__class__.__name__} {e}).")
except Exception as e:
    raise ValueError(f"An error with Ollama: ({e.__class__.__name__} {e}).")
    

In [ ]:
from core.Model import LLM, GenerationConfig, SupportedModels

llm = LLM(SupportedModels.Ollama.llama_32_1b)
cfg = GenerationConfig()

# system = "Your job is to extract the SQL query from this piece of text. Place your answer in ```sql ```"
system = "Correct the given SQL query to ensure it is valid SQLite"
user = """
```sql
SELECT 
    MAX(CreationDate) AS MaxConsumptionYear
FROM 
    yearmonth
WHERE 
    EXTRACT(YEAR FROM CreationDate) BETWEEN 2020 AND 2023;
```
"""

def make_msg(system_prompt, user_prompt):
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

reply = llm(make_msg(system, user), cfg)
print(reply)
llm.count_total_tokens()

In [ ]:
import json

with open('/home/fahim/Documents/sql-gen/results/llama3.2:1b_zeroshot-metaprompt-optimizer/mp_raw.json', 'r') as f:
    resp = json.load(f)

    for r in resp:
        print('_' * 25)
        print(r)
        print('_' * 25)
        print('\n\n')

# Run Experiment

## GPT-4o Zero-shot

In [26]:
if EXPERIMENT == 'zero-shot':
    print(f"Experiment: {MODEL}_{EXPERIMENT}")
    
    # Setup
    df, db_names = read_dataset()
    db_schemas   = fetch_BIRD_schemas(db_names)
    print(f'{db_names=}, {len(df)=}')
    
    client = get_openai_client()
    agent = ZeroShotAgent(MODEL, client, get_db_cursor, db_schemas, OUTPUT_PATH)
    evaluator = EvaluatorForBIRD(get_db_cursor)
    
    # Generate
    raw_responses = agent.batched_generate(df)
    dump_to_json('raw_responses', raw_responses)

    # Parse
    print("Finished Generating. Attempting SQL auto-parsing...")
    cleaned_sql = agent.auto_parse_sql_from_response(raw_responses)
    dump_to_json('cleaned_sql', cleaned_sql)
    print("SQL auto-parsing successful")

    # Evaluate
    df['prediction'] = cleaned_sql
    df['label'] = evaluator.evaluate(df, pred_col_name='prediction')
    
    # Save results
    df.to_json(OUTPUT_PATH / f'{MODEL}_{EXPERIMENT}_df.json', orient='records')

## GPT-4o Zero-shot + Optimizer

In [ ]:
if EXPERIMENT == 'optimizer-agent':
    print(f"Experiment: {MODEL}_{EXPERIMENT}")
    
    # Setup
    df, db_names = read_dataset()
    db_schemas   = fetch_BIRD_schemas(db_names)
    print(f'{db_names=}, {len(df)=}')
    
    client = get_openai_client()
    agent = OptimizerAgent(MODEL, client, get_db_cursor, db_schemas, OUTPUT_PATH)
    evaluator = EvaluatorForBIRD(get_db_cursor)
    
    # Generate
    df = pd.read_json('gpt-4o_zero-shot_df.json')
    raw_responses = agent.batched_generate(df)
    dump_to_json('raw_responses', raw_responses)

    # Parse
    print(f"Finished Generating. Attempting SQL auto-parsing...")
    cleaned_sql = agent.auto_parse_sql_from_response(raw_responses)
    dump_to_json('cleaned_sql', cleaned_sql)
    print(f"SQL auto-parsing successful")

    # Evaluate
    df['optimized'] = cleaned_sql
    df['opt-label'] = evaluator.evaluate(df, pred_col_name='optimized')
    
    # Save results
    df.to_json(OUTPUT_PATH / f'{MODEL}_{EXPERIMENT}_df.json', orient='records')

## GPT-4o Multi-Agent Discussion

In [ ]:
if EXPERIMENT == 'discussion':
    print(f"Experiment: {MODEL}_{EXPERIMENT}")
    
    # Setup
    df, db_names = read_dataset()
    db_schemas   = fetch_BIRD_schemas(db_names)
    print(f'{db_names=}, {len(df)=}')

    client = get_openai_client()
    multi_agent = MultiAgentDiscussion(MODEL, client, get_db_cursor, db_schemas, OUTPUT_PATH)
    evaluator = EvaluatorForBIRD(get_db_cursor)


    # Generate
    raw_responses = multi_agent.batched_generate(df, rounds=3)
    dump_to_json('raw_responses', raw_responses)

    # Parse
    print(f"Finished Generating. Attempting SQL auto-parse...")

    starter_zero = multi_agent.auto_parse_sql_from_response([response['agent_zero_shot'][0] for response in raw_responses])
    dump_to_json('cleaned_zeroshot_starter', starter_zero)

    starter_meta = multi_agent.auto_parse_sql_from_response([response['agent_meta_prompt'][0] for response in raw_responses])
    dump_to_json('cleaned_starter_meta', starter_meta)
    
    cleaned_sql  = multi_agent.auto_parse_sql_from_response([response['verdict'] for response in raw_responses])
    dump_to_json('cleaned_sql', cleaned_sql)

    print(f"SQL auto-parsing successful\n\n")


    # Evaluate results
    print("Evaluating Zero-shot starter generated queries")
    df['starter_zero_shot'] = starter_zero
    df['zero_shot_labels']  = evaluator.evaluate(df, pred_col_name='starter_zero_shot')

    print("Evaluating meta-prompt starter generated queries")
    df['starter_meta_prompt'] = starter_meta
    df['meta_prompt_labels']  = evaluator.evaluate(df, pred_col_name='starter_meta_prompt')

    print("Evaluating Multi-Agent Discussion generated queries")
    df['prediction'] = cleaned_sql
    df['label']      = evaluator.evaluate(df, pred_col_name='prediction')


    # Save results
    df.to_json(OUTPUT_PATH / f'{MODEL}_{EXPERIMENT}_df.json', orient='records')

# Experiments:
- Zero Shot
    - with/without COT
- Optimizer (on top of zero-shot)
- Multi-agent:
    - Zero-shot -> Optimizer -> Multi-agent Debate
    - Zero-shot -> Optimizer -> Multi-agent Discussion
    - Best of the above -> Optimizer
- Decomposition and Generation via Multi-agent Debate/Discussion
- Sparse Topology Multi-agent Debate/Discussion
- Augmenting schema with LLM calls:
    - Point out relationships (graph idea)
    - Write short descriptions regarding tables, columns